## Class for problem according to the structure of AIMA textbook

In [1]:
class Problem:
    def __init__(self,initial_state,goal=None):
        self.initial_state=initial_state
        self.goal=goal
        
    def actions(self,state):
        indices=[]
        actions=[]
        for i in range(3):
            for j in range(3):
                if state[i][j]==0:
                    indices.append(i)
                    indices.append(j)
        if indices[0]+1<3:
            actions.append("Down")
        if indices[0]-1>=0:
            actions.append("Up")
        if indices[1]+1<3:
            actions.append("Right")
        if indices[1]-1>=0:
            actions.append("Left")
        return actions
    
    def result(self,state,action):
        indices=[]
        for i in range(3):
            for j in range(3):
                if state[i][j]==0:
                    indices.append(i)
                    indices.append(j)
        res=[[0]*3,[0]*3,[0]*3]
        for i in range(0,3):
            for j in range(0,3):
                res[i][j]=state[i][j]
        if action=="Up":
            res[indices[0]][indices[1]],res[indices[0]-1][indices[1]]=res[indices[0]-1][indices[1]],res[indices[0]][indices[1]]
        elif action=="Down":
            res[indices[0]][indices[1]],res[indices[0]+1][indices[1]]=res[indices[0]+1][indices[1]],res[indices[0]][indices[1]]
        elif action=="Left":
            res[indices[0]][indices[1]],res[indices[0]][indices[1]-1]=res[indices[0]][indices[1]-1],res[indices[0]][indices[1]]
        elif action=="Right":
            res[indices[0]][indices[1]],res[indices[0]][indices[1]+1]=res[indices[0]][indices[1]+1],res[indices[0]][indices[1]]
        else:
            return state
        return res
    
    
    def goalTest(self,state):
        if state==self.goal:
            return True
        else:
            return False
        
    def stepCost(self,state,action):
        return 1
        
    def pathCost(self,state1,action,state2):
        state2.pathCost=state1.pathcost+self.stepCost(state1,action)
        
        


## Class for a node to denote states

In [2]:
    class Node:
        def __init__(self,state,parent=None,action=None,path_cost=0,heuristic=0,fvalue=0):
            self.state=state
            self.parent=parent
            self.action=action
            self.path_cost=path_cost
            self.heuristic=heuristic
            self.fvalue=fvalue
            if parent!=None:
                self.path_cost=self.parent.path_cost+1

        def __lt__(self,other):
            return self.state<other.state

        def solution(self):
            sol=[]
            while self.parent!=None:
                sol.append(self.action)
                self=self.parent
            return sol


        def child_node(self,problem,action):
            state=problem.result(self.state,action)
            pc=self.path_cost+problem.stepCost(self.state,action)
            childNode=Node(state)
            childNode.parent=self
            childNode.action=action
            return childNode

        def find_heuristic(self,problem):
            for i in range(3):
                for j in range(3):
                    if self.state[i][j]!=problem.goal[i][j]:
                        self.heuristic+=1

        def find_fvalue(self,problem):
            self.find_heuristic(problem)
            self.fvalue=self.path_cost+self.heuristic



## Class for a simple problem solving agent. Here the agent is an 8-puzzle solver.

In [3]:
import heapq

In [4]:
def breadth_first_graph_search(problem):
    node1=Node(problem.initial_state)
    if problem.goalTest(node1.state):
        return node1.solution()[::-1]
    frontier=[]
    frontier.append(node1)
    explored_set=set()        
    while frontier:
        temp=frontier.pop(0)
        explored_set.add(temp)
        lis_act=problem.actions(temp.state)
        for act in lis_act:
            child=temp.child_node(problem,act)
            if child.state not in [node2.state for node2 in frontier] and child.state not in [node3.state for node3 in explored_set]:
                if problem.goalTest(child.state):
                    return child.solution()[::-1]
                frontier.append(child)
    

In [5]:
def depth_first_graph_search(problem):
    node1=Node(problem.initial_state)
    frontier=[]
    frontier.append(node1)
    explored_set=set()
    while frontier:
        temp=frontier.pop()
        if problem.goalTest(temp.state):
            return temp.solution()[::-1]
        explored_set.add(temp)
        lis_act=problem.actions(temp.state)
        for act in lis_act:
            child=temp.child_node(problem,act)
            if child.state not in [node2.state for node2 in frontier] and child.state not in [node3.state for node3 in explored_set]:
                frontier.append(child)
    

In [6]:
def greedy_best_first_search(problem):
    node1=Node(problem.initial_state)
    node1.find_heuristic(problem)
    frontier=[]
    heapq.heappush(frontier,(node1.heuristic,node1))
    explored_set=set()
    while frontier:
        temp=heapq.heappop(frontier)
        if problem.goalTest(temp[1].state):
            return temp[1].solution()
        explored_set.add(temp[1])
        lis_act=problem.actions(temp[1].state)
        for act in lis_act:
            child=temp[1].child_node(problem,act)
            child.find_heuristic(problem)
            if child.state not in [nodex[1].state for nodex in frontier] and child.state not in [noder.state for noder in explored_set]:
                heapq.heappush(frontier,(child.heuristic,child))


In [7]:
def A_star_algorithm(problem):
    node1=Node(problem.initial_state)
    node1.find_fvalue(problem)
    frontier=[]
    heapq.heappush(frontier,(node1.fvalue,node1))
    explored_set=set()
    while frontier:
        temp=heapq.heappop(frontier)
        if problem.goalTest(temp[1].state):
            return temp[1].solution()[::-1]
        explored_set.add(temp[1])
        lis_act=problem.actions(temp[1].state)
        for act in lis_act:
            child=temp[1].child_node(problem,act)
            child.find_fvalue(problem)
            if child.state not in [nodex[1].state for nodex in frontier] and child.state not in [noder.state for noder in explored_set]:
                heapq.heappush(frontier,(child.fvalue,child))
    

In [8]:
class SimpleProblemSolvingAgent:
    def __init__(self,initial_state=None):
        self.initial_state=initial_state
        self.seq=[]
        
    def update_state(self,percept):
        return percept
    
    def formulate_goal(self,state):
        return [[0,1,2],[3,4,5],[6,7,8]]
    
    def formulate_problem(self,goal):
        return Problem(self.initial_state,goal)
    
    def search(self,problem,mode):
        if mode=="BFS":
            sequence=breadth_first_graph_search(problem)
        elif mode=="DFS":
            sequence=depth_first_graph_search(problem)
        elif mode=="GBFS":
            sequence=greedy_best_first_search(problem)
        elif mode=="A*":
            sequence=A_star_algorithm(problem)
        else:
            print("Invalid mode!")
        return sequence
            
    def solve(self,percept):
        state=self.update_state(percept)
        if not self.seq:
            goal=self.formulate_goal(state)
            problem=self.formulate_problem(goal)
            self.seq=self.search(problem,"BFS")
            if not self.seq:
                return None
        return self.seq



In [9]:
# state=[[7,2,4],[5,0,6],[8,3,1]]
state=[[1,0,2],[3,4,5],[6,7,8]]
agent=SimpleProblemSolvingAgent(state)
lis=agent.solve(state)
print(lis)

['Left']
